In [1]:
import pandas as pd
import argparse
import os
import sklearn.metrics
from aggregate_entity_results import aggregate_results
from utils import generate_explanation_prediction
import json

In [49]:
size = 1.0
version_str = "lv_v1_tv_v3"
agg_method = "expert"
model_name = "gpt-3.5-turbo"
few_shot_selection = "label_diversity_similarity"
temperature = 0.1
verified = False
explanation_column = "two_step"
check_mode = "strict"
CoT = True
verify_CoT = False
use_current_explanation = False
use_ground_explanation = False
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.traintestfortest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-' + agg_method
label_symbol = "^^^^"
text_column = "context"
result_dir = '/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-' + agg_method
output_dir = os.path.join(output_dir, str(temperature), str(size), version_str)
result_dir = os.path.join(result_dir, str(temperature), str(size), version_str)

In [50]:
output_df_list = []
for label_type in ['Food', 'Location', 'Symptom', 'Keyword']:
    if explanation_column == "two_step":
        label_explanation_column = "Two_step_" + label_type + "_explanation"
    else:
        label_explanation_column = label_type + "_explanation"
    if verified:
        final_name = '_final_COT_' if verify_CoT else '_final_'
        if CoT:
            agg_output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + str(label_explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + final_name + os.path.split(data_path)[-1]
        else:
            agg_output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + label_type + final_name + os.path.split(data_path)[-1]
    else:
        if CoT:
            agg_output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + str(label_explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_' + os.path.split(data_path)[-1]
        else:
            agg_output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + label_type + '_' + os.path.split(data_path)[-1]
    output_path = os.path.join(result_dir, agg_output_name)
    print(output_path)
    output_df = pd.read_csv(output_path)
    output_df_list.append(output_df)

/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-expert/0.1/1.0/lv_v1_tv_v3/gpt-3.5-turbo_label_diversity_similarity_COT_Two_step_Food_explanation_cur_False_ground_False_Food_expert.traintestfortest.csv
/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-expert/0.1/1.0/lv_v1_tv_v3/gpt-3.5-turbo_label_diversity_similarity_COT_Two_step_Location_explanation_cur_False_ground_False_Location_expert.traintestfortest.csv
/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-expert/0.1/1.0/lv_v1_tv_v3/gpt-3.5-turbo_label_diversity_similarity_COT_Two_step_Symptom_explanation_cur_False_ground_False_Symptom_expert.traintestfortest.csv
/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-expert/0.1/1.0/lv_v1_tv_v3/gpt-3.5-turbo_label_diversity_similarity_COT_Two_step_Keyword_explanation_cur_False_ground_False_Keyword_expert.traintestfortest.csv


In [51]:
output_df[f"{label_type}_answer_annotation"].loc[output_df[f"{label_type}_answer_error"].notna()].index

Index([   3,    7,   14,   21,   41,   55,   79,  151,  159,  160,
       ...
       3492, 3512, 3524, 3579, 3610, 3629, 3644, 3646, 3648, 3662],
      dtype='int64', length=256)

In [52]:
output_df[f'{label_type}_answer_prompt'][0]

'You are an expert at extracting Keyword entities that are related to foodborne illness incident from text. In the given text, your task is to label Keyword entities that include other relevant keywords or terms associated with foodborne illnesses, such as "food poisoning". If a Keyword entity is not associated with a potential foodborne illnesses incident, the entity should not be labeled as relevant entities.\n\nYour answer will consist of an explanation, followed by the correct labeled sentence. In the last line of the response, you should present the input text with Keyword entities related to foodborne illness incidents distinctly marked. To highlight these Keyword entities, prepend each with "^^" and append "^^". Refer to the provided examples to structure your predictions correctly.\n\nSome examples with their output answers are provided below:\n\nInput: My Stomach Ughh Food Poison : (\nOutput: Let\'s think step by step.\nThe text indicates a foodborne illness incident. The word

In [45]:
output_df[f"{label_type}_answer_annotation"].loc[output_df[f"{label_type}_answer_error"].notna()][3]

'successfully_labeled=False label=\'NO_LABEL\' curr_sample=b"\\x80\\x04\\x95\\xf9\\x00\\x00\\x00\\x00\\x00\\x00\\x00}\\x94(\\x8c\\x07context\\x94\\x8c`@USER @USER they think its food poisoning . She\'s home already thank God . I\'ll keep you updated\\x94\\x8c\\x0bhas_Keyword\\x94\\x8c\\x02no\\x94\\x8c\\x0eKeyword_answer\\x94\\x8c`@USER @USER they think its food poisoning . She\'s home already thank God . I\'ll keep you updated\\x94u." confidence_score=None generation_info={\'finish_reason\': \'stop\', \'logprobs\': None} raw_response=\'The text indicates a foodborne illness incident. The phrase "food poisoning" in the text is a Keyword entity, which is other relevant keyword or term associated with foodborne illnesses, such as "food poisoning".\\n{"label": "@USER @USER they think its ^^food poisoning^^ . She\\\'s home already thank God . I\\\'ll keep you updated"}\\n\\nInput text with Keyword entities related to foodborne illness incidents distinctly marked:\\n@USER @USER they think it

In [30]:
output_df["Keyword_answer"][3]

"@USER @USER they think its food poisoning . She's home already thank God . I'll keep you updated"

In [ ]:
output_df["Keyword_answer"][3]

In [31]:
output_df["context"][3]

"@USER @USER they think its food poisoning . She's home already thank God . I'll keep you updated"

In [5]:
output_df.columns

Index(['id', 'tweet', 'tweet_tokens', 'entity_label', 'related_label',
       'sentence_class', 'relevant_entity_label',
       'relevance_entity_class_label', 'real_relevant_entity_label',
       'IndividualLabels', 'CategorizedLabels', 'Food_answer',
       'Location_answer', 'Symptom_answer', 'Keyword_answer', 'has_Food',
       'has_Location', 'has_Symptom', 'has_Keyword', 'has_all', 'context',
       'sentence_explanation', 'word_explanation',
       'CategorizedLabels_prediction', 'sentence_explanation_prediction'],
      dtype='object')

In [6]:
output_df['has_Food'].value_counts()

has_Food
no     3338
yes     372
Name: count, dtype: int64

In [7]:
output_df['has_all'].value_counts()

has_all
no     2485
yes    1225
Name: count, dtype: int64

In [8]:
word_pred = ~output_df['sentence_explanation_prediction'].str.startswith('No')

In [9]:
word_label = ~output_df[['CategorizedLabels', 'sentence_class']].apply(lambda x: generate_explanation_prediction(x['CategorizedLabels'], x['sentence_class']), axis=1).str.startswith('No')

In [10]:
def count_entities(label_dict):
    label_dict = json.loads(label_dict)
    cnt = 0
    for key, val in label_dict.items():
        cnt += len(val)
    return cnt

In [11]:
output_df['CategorizedLabels'].apply(count_entities).sum()

2648

In [12]:
output_df['CategorizedLabels_prediction'].apply(count_entities).sum()

1861

In [13]:
label = output_df['sentence_class'] == 'Yes'

In [14]:
sklearn.metrics.accuracy_score(label, word_pred)

0.7466307277628033

In [15]:
sklearn.metrics.f1_score(label, word_pred)

0.637904468412943

In [16]:
sklearn.metrics.precision_score(label, word_pred)

0.6039387308533917

In [17]:
sklearn.metrics.recall_score(label, word_pred)

0.6759183673469388

In [18]:
sklearn.metrics.accuracy_score(label, word_label)

1.0

In [19]:
sklearn.metrics.f1_score(label, word_label)

1.0